# Creating Final Dataset for Machine Learning

### Import Libraries and Set Configuration

In [65]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
from datetime import datetime

# Set plotting style
plt.style.use('seaborn-v0_8')
%matplotlib inline

### Load All Datasets

In [66]:
print("LOADING AND PREPROCESSING DATASETS")
print("=" * 60)

# Load main datasets
art_price_df = pd.read_csv('../data/processed/artDataset.csv')
artists_df = pd.read_csv('../data/processed/artists.csv')
artists_bat_df = pd.read_csv('../data/processed/artistsBAT.csv')
artworks_df = pd.read_csv('../data/processed/artworks.csv')
market_df = pd.read_csv('../data/processed/all_artists_artworks.csv')

# Display initial shapes
print("Initial dataset shapes:")
print(f"Art Price Dataset: {art_price_df.shape}")
print(f"Artists Dataset: {artists_df.shape}")
print(f"Best Artworks of All Time Dataset: {artists_bat_df.shape}")
print(f"Artworks Dataset: {artworks_df.shape}")
print(f"Million Dollar Art Market Dataset: {market_df.shape}")

LOADING AND PREPROCESSING DATASETS
Initial dataset shapes:
Art Price Dataset: (754, 9)
Artists Dataset: (15091, 6)
Best Artworks of All Time Dataset: (50, 8)
Artworks Dataset: (130262, 21)
Million Dollar Art Market Dataset: (1144, 10)


### Step 1: Preprocess Art Price Dataset (Main Base)

In [67]:
print("\n1. PREPROCESSING ART PRICE DATASET")
print("-" * 40)

# Remove unnecessary columns
columns_to_drop = []
if 'Unnamed: 0' in art_price_df.columns:
    columns_to_drop.append('Unnamed: 0')
if 'condition' in art_price_df.columns:
    columns_to_drop.append('condition')

if columns_to_drop:
    art_price_df = art_price_df.drop(columns_to_drop, axis=1)
    print(f"✓ Removed columns: {columns_to_drop}")

# Clean price column and convert to numeric
def extract_price(price_str):
    """Extract numeric price from string format like '28.500 USD'"""
    if pd.isna(price_str):
        return np.nan
    # Remove currency symbols and text, keep only numbers and dots
    price_clean = re.sub(r'[^\d.]', '', str(price_str))
    try:
        return float(price_clean) if price_clean else np.nan
    except:
        return np.nan

art_price_df['price_usd'] = art_price_df['price'].apply(extract_price)
art_price_df = art_price_df.drop('price', axis=1)
print("✓ Converted price to numeric format")

# Extract year from yearCreation column
def extract_year(year_str):
    """Extract year from various formats including textual descriptions"""
    if pd.isna(year_str):
        return np.nan
    
    year_str = str(year_str).strip()
    
    # First try to extract exact 4-digit years
    year_match = re.search(r'\b(1[0-9]{3}|20[0-9]{2})\b', year_str)
    if year_match:
        return int(year_match.group())
    
    year_str_lower = year_str.lower()
    
    # Map textual descriptions to approximate years
    period_mapping = {
        'first half 20th century': 1925,  # Mid-point of 1900-1950
        'second half 20th century': 1975,  # Mid-point of 1950-2000
        'early 20th century': 1910,
        'mid 20th century': 1950,
        'late 20th century': 1980,
        '19th century': 1850,
        '18th century': 1750,
        'contemporary': 2010,  # Approximate for contemporary works
    }
    
    for pattern, approx_year in period_mapping.items():
        if pattern in year_str_lower:
            return approx_year
    
    # If no pattern matches, return NaN (I'll handle missing values later)
    return np.nan

art_price_df['creation_year'] = art_price_df['yearCreation'].apply(extract_year)
art_price_df = art_price_df.drop('yearCreation', axis=1)
print("✓ Extracted creation year from text")

def categorize_signature(signed_str):
    if pd.isna(signed_str):
        return 0
    return 1

art_price_df['signed'] = art_price_df['signed'].str.strip().replace('[nan]', np.nan)
art_price_df['signed_binary'] = art_price_df['signed'].apply(categorize_signature)
art_price_df = art_price_df.drop('signed', axis=1)

art_price_df['period'] = art_price_df['period'].str.strip().replace('[nan]', np.nan)

# Clean movement column
art_price_df['movement'] = art_price_df['movement'].str.strip()
print("✓ Cleaned movement column")

print(f"\nProcessed art price dataset: {art_price_df.shape}")
print(f"Price range: ${art_price_df['price_usd'].min():,.0f} - ${art_price_df['price_usd'].max():,.0f}")

print("\nFirst 3 rows of processed data:")
display(art_price_df.head(3))


1. PREPROCESSING ART PRICE DATASET
----------------------------------------
✓ Removed columns: ['Unnamed: 0', 'condition']
✓ Converted price to numeric format
✓ Extracted creation year from text
✓ Cleaned movement column

Processed art price dataset: (754, 7)
Price range: $1 - $995

First 3 rows of processed data:


,artist,title,period,movement,price_usd,creation_year,signed_binary
0,Tommaso Ottieri,Bayreuth Opera,Contemporary,Baroque,28.5,2021.0,1
1,Pavel Tchelitchew,Drawings of the Opera,Post-War,Surrealism,3.0,1925.0,1
2,Leo Gabin,Two on Sidewalk,Contemporary,Abstract,5.0,2016.0,1


### Step 2: Preprocess Artists Dataset

In [68]:
print("\n2. PREPROCESSING ARTISTS DATASET")
print("-" * 40)

# Clean column names and standardize
artists_df.columns = [col.strip().lower().replace(' ', '_') for col in artists_df.columns]
print("✓ Standardized column names")

# Keep only essential columns for merging
artists_df_clean = artists_df[['artist_id', 'name']].copy()

# Clean artist names for consistent merging
artists_df_clean['name'] = artists_df_clean['name'].str.strip().str.title()

print(f"✓ Kept only essential columns: {list(artists_df_clean.columns)}")
print(f"Processed artists dataset: {artists_df_clean.shape}")

print("\nArtist dataset sample:")
display(artists_df_clean.head(3))


2. PREPROCESSING ARTISTS DATASET
----------------------------------------
✓ Standardized column names
✓ Kept only essential columns: ['artist_id', 'name']
Processed artists dataset: (15091, 2)

Artist dataset sample:


,artist_id,name
0,1,Robert Arneson
1,2,Doroteo Arnaiz
2,3,Bill Arnold


### Step 3: Preprocess Artists BAT Dataset

In [69]:
print("\n3. PREPROCESSING ARTISTS BAT DATASET")
print("-" * 40)

# Extract birth and death years from 'years' column
def extract_lifespan(years_str):
    """Extract birth and death years from string like '1884 - 1920'"""
    if pd.isna(years_str):
        return np.nan, np.nan
    years_match = re.findall(r'\b(1[0-9]{3}|20[0-9]{2})\b', str(years_str))
    if len(years_match) >= 2:
        return int(years_match[0]), int(years_match[1])
    elif len(years_match) == 1:
        return int(years_match[0]), np.nan
    return np.nan, np.nan

artists_bat_df[['birth_year', 'death_year']] = pd.DataFrame(
    artists_bat_df['years'].apply(extract_lifespan).tolist(), 
    index=artists_bat_df.index
)
print("✓ Extracted birth and death years from lifespan string")

# Calculate artist lifespan (only if both dates are available)
def calculate_lifespan(birth_year, death_year):
    """Calculate lifespan only if both years are available"""
    if pd.isna(birth_year) or pd.isna(death_year):
        return np.nan
    return death_year - birth_year

artists_bat_df['lifespan'] = artists_bat_df.apply(
    lambda row: calculate_lifespan(row['birth_year'], row['death_year']), 
    axis=1
)

# Calculate years since death (only for deceased artists)
def calculate_years_since_death(death_year):
    """Calculate years since death only if death year is available"""
    if pd.isna(death_year):
        return np.nan
    return datetime.now().year - death_year

artists_bat_df['years_since_death'] = artists_bat_df['death_year'].apply(calculate_years_since_death)
artists_bat_df['is_living'] = artists_bat_df['death_year'].isna()

print("✓ Calculated lifespan and historical context")

# Clean and standardize names for merging
artists_bat_df['name'] = artists_bat_df['name'].str.strip().str.title()
print("✓ Standardized artist names for merging")

# Select only useful columns for final dataset
useful_columns = [
    'name', 'lifespan', 'years_since_death', 'is_living', 'paintings'
]
artists_bat_clean = artists_bat_df[useful_columns].copy()

print(f"\nProcessed artists BAT dataset: {artists_bat_clean.shape}")
print("\nArtists BAT sample:")
display(artists_bat_clean.head(3))


3. PREPROCESSING ARTISTS BAT DATASET
----------------------------------------
✓ Extracted birth and death years from lifespan string
✓ Calculated lifespan and historical context
✓ Standardized artist names for merging

Processed artists BAT dataset: (50, 5)

Artists BAT sample:


,name,lifespan,years_since_death,is_living,paintings
0,Amedeo Modigliani,36,105,False,193
1,Vasiliy Kandinskiy,78,81,False,88
2,Diego Rivera,71,68,False,70


### Step 4: Preprocess Artworks Dataset for Feature Extraction

In [70]:
print("\n4. PREPROCESSING ARTWORKS DATASET")
print("-" * 40)

# Clean column names
artworks_df.columns = [col.strip().lower().replace(' ', '_') for col in artworks_df.columns]
print("✓ Standardized column names")

# Extract dimensions and create size categories
def extract_avg_dimension(dimensions_str):
    """Extract average dimension from string format"""
    if pd.isna(dimensions_str):
        return np.nan
    
    # Look for measurements in cm
    cm_matches = re.findall(r'(\d+\.?\d*)\s*cm', str(dimensions_str), re.IGNORECASE)
    if cm_matches:
        numeric_dims = [float(x) for x in cm_matches]
        return np.mean(numeric_dims) if numeric_dims else np.nan
    
    # Look for measurements in inches
    inch_matches = re.findall(r'(\d+\.?\d*)\s*"', str(dimensions_str))
    if inch_matches:
        numeric_dims = [float(x) * 2.54 for x in inch_matches]  # Convert to cm
        return np.mean(numeric_dims) if numeric_dims else np.nan
    
    return np.nan

artworks_df['avg_dimension_cm'] = artworks_df['dimensions'].apply(extract_avg_dimension)
print("✓ Extracted average dimension from text")

# Create size categories
def categorize_size(avg_dim):
    if pd.isna(avg_dim):
        return 'Unknown'
    elif avg_dim < 30:
        return 'Small'
    elif avg_dim < 100:
        return 'Medium'
    else:
        return 'Large'

artworks_df['size_category'] = artworks_df['avg_dimension_cm'].apply(categorize_size)
print("✓ Created size categories")

essential_columns = [
    'artist_id',      
    'title',         
    'avg_dimension_cm',
    'size_category'   
]

artworks_df = artworks_df[essential_columns].copy()
print("✓ Kept only essential columns")

print(f"\nProcessed artworks dataset: {artworks_df.shape}")
print("\nArtworks data sample:")
display(artworks_df.head(10))


4. PREPROCESSING ARTWORKS DATASET
----------------------------------------
✓ Standardized column names


✓ Extracted average dimension from text
✓ Created size categories
✓ Kept only essential columns

Processed artworks dataset: (130262, 4)

Artworks data sample:


,artist_id,title,avg_dimension_cm,size_category
0,6210,"Ferdinandsbrücke Project, Vienna, Austria, Ele...",168.9,Large
1,7470,"City of Music, National Superior Conservatory ...",29.8,Small
2,7605,"Villa near Vienna Project, Outside Vienna, Aus...",31.8,Medium
3,7056,"The Manhattan Transcripts Project, New York, N...",50.8,Medium
4,7605,"Villa, project, outside Vienna, Austria, Exter...",19.1,Small
5,7056,"The Manhattan Transcripts Project, New York, N...",45.7,Medium
6,7056,"The Manhattan Transcripts Project, New York, N...",45.7,Medium
7,7056,"The Manhattan Transcripts Project, New York, N...",45.7,Medium
8,7056,"The Manhattan Transcripts Project, New York, N...",45.7,Medium
9,7056,"The Manhattan Transcripts Project, New York, N...",45.7,Medium


### Step 5: Preprocess Market Dataset for Additional Features

In [71]:
print("\n5. PREPROCESSING MARKET DATASET")
print("-" * 40)

# Clean purchase and sale prices
def clean_market_price(price_str):
    """Convert market prices like '$2.2K' to numeric USD"""
    if pd.isna(price_str):
        return np.nan
    
    price_str = str(price_str).upper().replace('$', '').replace(',', '')
    
    # Handle K, M suffixes
    multiplier = 1
    if 'K' in price_str:
        multiplier = 1000
        price_str = price_str.replace('K', '')
    elif 'M' in price_str:
        multiplier = 1000000
        price_str = price_str.replace('M', '')
    
    try:
        return float(price_str) * multiplier
    except:
        return np.nan

market_df['purchase_price_usd'] = market_df['purchase_price'].apply(clean_market_price)
market_df['sale_price_usd'] = market_df['sale_price'].apply(clean_market_price)
print("✓ Converted market prices to numeric format")

# Calculate appreciation multiplier if both prices available
market_df['appreciation_multiplier'] = np.where(
    (market_df['purchase_price_usd'].notna()) & (market_df['sale_price_usd'].notna()) & (market_df['purchase_price_usd'] > 0),
    market_df['sale_price_usd'] / market_df['purchase_price_usd'],
    np.nan
)
print("✓ Calculated appreciation multipliers")

# Clean artist names for merging
market_df['artist'] = market_df['artist'].str.strip().str.title()
print("✓ Standardized artist names for merging")

columns_to_drop = [
    'description',
    'purchase_price',
    'sale_price',
    'gross_appreciation_multiplier',
    'gross_appreciation_period',
    'url',
    'has_image',
    'image_url'
]

market_df_clean = market_df.drop(columns=columns_to_drop, errors='ignore')
print("✓ Removed unnecessary columns")

print(f"\nProcessed market dataset: {market_df_clean.shape}")
print("\nMarket data sample:")
market_sample_cols = ['artist', 'title', 'purchase_price_usd', 'sale_price_usd', 'appreciation_multiplier']
display(market_df_clean.head(3))


5. PREPROCESSING MARKET DATASET
----------------------------------------
✓ Converted market prices to numeric format
✓ Calculated appreciation multipliers
✓ Standardized artist names for merging
✓ Removed unnecessary columns

Processed market dataset: (1144, 5)

Market data sample:


,title,artist,purchase_price_usd,sale_price_usd,appreciation_multiplier
0,Four jackies,Andy Warhol,2200.0,1900000.0,863.636364
1,Lavender marilyn (1),Andy Warhol,7200.0,4600000.0,638.888889
2,Self-portrait,Andy Warhol,31200.0,5700000.0,182.692308


### Step 6: Create Final Merged Dataset

In [72]:
print("\n6. CREATING FINAL MERGED DATASET")
print("-" * 40)

# Start with art price dataset as base
final_df = art_price_df.copy()
print(f"✓ Using art price dataset as base: {final_df.shape}")

final_df['artist'] = final_df['artist'].str.strip().str.title()

# MERGE 1: Add artist_id from artists dataset
final_df = final_df.merge(
    artists_df_clean[['name', 'artist_id']],
    left_on='artist',
    right_on='name',
    how='left'
)
print("✓ Merged artist_id from artists dataset")

final_df['artist_id'] = final_df['artist_id'].astype(str)
artworks_df['artist_id'] = artworks_df['artist_id'].astype(str)

# MERGE 2: Add dimensions from artworks dataset
artworks_aggregated = artworks_df.groupby('artist_id').agg({
    'avg_dimension_cm': 'mean',
    'size_category': lambda x: x.mode()[0] if not x.mode().empty else np.nan
}).reset_index()

final_df = final_df.merge(
    artworks_aggregated,
    on='artist_id',
    how='left'
)
print("✓ Merged dimension data from artworks dataset")

# MERGE 3: Add artist info from BAT dataset
final_df = final_df.merge(
    artists_bat_clean[['name', 'lifespan', 'years_since_death', 'is_living', 'paintings']],
    left_on='artist',
    right_on='name',
    how='left'
)
print("✓ Merged artist info from BAT dataset")

# MERGE 4: Add market data (aggregated by artist)
common_artists = set(final_df['artist']).intersection(set(market_df_clean['artist']))
print(f"Common artists between datasets: {len(common_artists)}")

artist_market_stats = market_df_clean.groupby('artist').agg({
    'purchase_price_usd': 'mean',
    'sale_price_usd': 'mean',
    'appreciation_multiplier': 'mean'
}).reset_index()

final_df = final_df.merge(
    artist_market_stats,
    on='artist',
    how='left'
)
print("✓ Merged market statistics")

# Create artist popularity based on MoMA collection
artist_moma_count = artworks_df['artist_id'].value_counts().reset_index()
artist_moma_count.columns = ['artist_id', 'moma_artwork_count']
artist_moma_count['artist_id'] = artist_moma_count['artist_id'].astype(str)

print("✓ Created additional feature")


6. CREATING FINAL MERGED DATASET
----------------------------------------
✓ Using art price dataset as base: (754, 7)
✓ Merged artist_id from artists dataset
✓ Merged dimension data from artworks dataset
✓ Merged artist info from BAT dataset
Common artists between datasets: 0
✓ Merged market statistics
✓ Created additional feature


In [73]:
final_columns = [
    # Core features
    'artist_id', 'artist', 'title', 'price_usd', 'creation_year',
    
    # Categorical features (for one-hot encoding)
    'period', 'movement', 'size_category',
    
    # Numerical features (for scaling)
    'avg_dimension_cm', 'lifespan', 'years_since_death', 'paintings',
    'avg_purchase_price', 'avg_sale_price', 'avg_appreciation', 
    'moma_artwork_count',
    
    # Binary features
    'signed_binary', 'is_living'
]

available_columns = [col for col in final_columns if col in final_df.columns]
final_df = final_df[available_columns]

print(f"✓ Final dataset shape: {final_df.shape}")
print(final_df.info())

base_path = "../data/processed/"
output_path = os.path.join(base_path, "final_art_dataset.csv")

final_df.to_csv(output_path, index=False)

✓ Final dataset shape: (754, 14)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 754 entries, 0 to 753
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   artist_id          754 non-null    object 
 1   artist             753 non-null    object 
 2   title              754 non-null    object 
 3   price_usd          754 non-null    float64
 4   creation_year      725 non-null    float64
 5   period             753 non-null    object 
 6   movement           754 non-null    object 
 7   size_category      458 non-null    object 
 8   avg_dimension_cm   458 non-null    float64
 9   lifespan           1 non-null      float64
 10  years_since_death  1 non-null      float64
 11  paintings          1 non-null      float64
 12  signed_binary      754 non-null    int64  
 13  is_living          1 non-null      object 
dtypes: float64(6), int64(1), object(7)
memory usage: 82.6+ KB
None
